An Evening with… EM!
===

The EM algorithm ([Dempster/Laird/Rubin, 1977](#refDempster)) is one of the most widely used unsupervised learning methods in NLP. It is important to have a solid understanding of its properties and limitations in order to use it well. However, it is quite complex and a bit tricky, and can easily be confusing for the beginner. There are a lot of terms, a lot of implementations, and a lot of Greek letters to add to the confusion. This tutorial tries to avoid the confusion by providing the step-by-step implementation of a concrete case (with pseudocode and a [Python implementation](#secrunnable)), highlighting only the necessary terms and providing layman’s explanations of the underlying concepts.

Table of contents
--
1. [Introduction](#secintro)
2. [Preliminaries](#secpreliminaries)
3. [Example Uses of EM](#secexample)
4. [The Goal](#secgoal)
5. [Implementation](#secimplementation)
6. [Finally...](#secfinally)
7. [Troubleshotting](#sectrouble)
8. [Useful Reading](#secreading)
9. [Advanced Topics](#secadvanced)
10. [Going Further...](#secfurther)


+ [Acknowledgements](#secacknowledgements)
+ [References](#secref)
+ [Answers](#secanswers)

In [12]:
import scipy as sp
from collections import defaultdict
import time
from functools import reduce

sp.set_printoptions( precision = 3 )

# CONSTANTS
NINF = float('-1e303')
LINF = sp.log(NINF)
SMOOTHING = sp.log(0.1)
HARDNESS = 1.0/1.0
THRESHOLD = 0.00001
MAX_NUM_ITERATIONS = 40
logadd = sp.logaddexp

1 Introduction<a name="secintro"></a>
==

Before we look at what EM is, let’s look at some of the common misconceptions about EM, and clear up what it is not: 
- a model (it’s a way to optimize one)
- a silver bullet (you have to be careful what to use it for)
- magic (really, it’s just code. . . )

So, after we got that out of the way, let’s see what EM actually is. EM is a form of ***unsupervised learning***. It is not so much an algorithm, but rather a class of algorithms that use a 2-step procedure learning (E step, M step) to train a ***generative model***. Generative models are joint probabilities (written $P(x,y)$) that explain how the data was, well, generated (*discriminative* models, on the other hand, are simply weight vectors that explain the conditional probability $P(x|y)$). EM tries to find the parameters of a model which best explain the observed data. Or: "If only I knew X, I could estimate Y. If only I knew Y, I could estimate X." We will see this sentence a couple of times.

Throughout the tutorial, there will be little questions to see whether you are still awake. You can just ignore them and read on, or try to solve them and feel good when you look up the answer at the end.
Don’t beat yourself up when you got one wrong, though!

Also, do not get scared by complicated-looking formulas. We will translate them into plain English, and the mathematical notation will become just the shorthand it was meant to be.

2 Preliminaries<a name="secpreliminaries"></a>
==

In order to understand EM, we need to look at probabilities and graphical models. If you are not quite sure what they are, fear not, we will explain them in the next sections. If you are already familiar with the concepts, you can skip ahead to [Section 3](#section3).

2.1 Probabilities
--

We will be talking about probabilities a lot, so we will review some basics. If you want to get deeper into it, look at the relevant sections in [Manning/Schütze (2000)](#refMS) and [Russell/Norvig (2003)](#refRN).
Since we do NLP, let’s look at the probability of words. Say we have a corpus of 100 sentences, $x$ is "unsupervised" and occurs in 20 sentences, and y is "learning" and occurs in 50 sentences.
Probabilities are basically counts that have been normalized. $P(x)$, or the probability of seeing a sentence with "unsupervised", is simply the count of sentences with "unsupervised" (20) divided by the number of all sentences in our corpus/text (100).

$$
P(x="unsupervised") = \frac{count(sentences\ with\ "unsupervised")}{number\ of\ all\ sentences} = \frac{20}{100}
= \frac{1}{5}
= 0.2
$$

P(x,y) is a joint probability, i.e., how likely is it that we see x and y together, that is the words in one sentence ("unsupervised learning" or "learning unsupervised", or even separated by other words, order does not matter in this case). Say we have 10 sentences that contain both words, then

$$
P(unsupervised, learning) =
\frac{count(sentences\ with\ "unsupervised"\ and\ "learning")}{number\ of\ all\ sentences}
= \frac{10}{100}
= \frac{1}{10}
= 0.1
$$

$P(y|x)$ is a conditional probability, i.e., how likely is it to see $y$ after having seen $x$, or "learning" in a sentence that contains "unsupervised". We can compute this as

$$
P(y|x) =
\frac{P(x,y)}{P(x)} =
\frac{\frac{count(sentences\ with\ "unsupervised"\ and\ "learning")}{number\ of\ all\ sentences}}
{\frac{count(sentences\ with\ "unsupervised")}{number\ of\ all\ sentences}}
= \frac{count(sentences\ with\ "unsupervised"\ and\ "learning")}{count(sentences\ with\ "unsupervised")}
= \frac{10}{20}
= \frac{1}{2}
= 0.5
$$

Note that order ***does*** matter in this case!

<font color="blue"><a name="q1"></a>**QUESTION**: What would be the corresponding probability formulation for seeing "unsupervised" in a sentence that contains "learning"?[$^1$](#answer1)</font>

<a name="secprob"></a>2.1.1 Where Probabilities Come From
--

You might have wondered where the probabilities we are gonna use come from. That is a very good
question, and there are two answers. The good, scientifically satisfying answer is “data". We can use
the statistics collected by statistical institutions (now you know what they are good for), or we can
get them by counting and dividing from (preferably large amounts of) data. How likely is ist that it is
sunny in LA? Get the weather data of the last 30 years, count the days marked *sunny* and divide by the
total number of days. The larger your sample, the more accurate your probability.

The other, less satisfying answer is “elaborate guessing". Some things cannot be measured, or there
is simply no data (some things are just too rare. The only exception to this rule is baseball. There are
statistics for everything in baseball). What is the probability of being eaten by a tiger in Montana?
Probably very small (luckily), but what number is “very small"? We can make up a number, but it
might not be accurate.

The good news is that we can use EM to make up numbers and then adjust them until our model
explains the data best.

2.2 Graphical Models
--

Graphical Models are a nice way of visualizing probabilistic models, and also to express the dependencies
that hold between the individual elements. There are several types of graphical models, but the
ones we are interested in (and the ones we mean when we use the term) are ***Bayes Nets*** and ***Hidden
Markov Models*** (HMMs). Graphical models consist of two elements, ***nodes*** and ***arcs***.

The nodes are ***random variables***. Random variables are events that have some probability, and come
in different flavors. If a random variable has exactly two values, like {`on`, `off`} or {`true`, `false`}, they
are ***binary*** (in the latter case ***boolean***). If they have a list of values (something like {`red`, `green`, `blue`}
or {`chocolate`, `vanilla`, `strawberry`, `pistacchio`}), they are ***discrete***. If they have numbers as values,
they are called ***continuous***. 
The ***probabilities*** associated with each of the values of a random variable have to
sum up to $1.0$, i.e., the variable `COLOR` with the values {`red`, `green`, `blue`} could have the respective probabilities {$0.2, 0.5, 0.3$} or
{$0.33, 0.33, 0.33$} associated with the values, but not {$0.8, 0.4, 0.7$}.

Arcs are the directed links between the random variables, and you can think of them as causal relations
(there are other kinds, but it is easiest this way). They denote what influence the parent has on the
child node. This influence is expressed by a conditional probability. 

<img src="pics/bn1.png" width="200px"/>
<div align="center">*Figure 1: A simple graphical model with three random variables*</div>

E.g., in a network like the one in Figure 1, we can say how likely it is that traffic ($T$) is bad, given that the weather ($W$) is rain. A node
$X$ can have several parents, which means that its value is influenced by several factors (traffic could
also be influenced by a Lakers game, $G$). If there are no links between two variables, then they are
***independent of one another***, i.e., whether the Lakers play or not luckily has no influence on the weather
$W$ (the examples in this section are largely influenced by [Russell/Norvig 2003](#refRN)).

2.2.1 Bayes Nets
--
<img src="pics/bn2.png" width="450px"/>
<div align="center">*Figure 2: A Bayes Net with three random variables and associated parameters*</div>

If we combine several nodes in a network, we call it a ***Bayes Net***. Let’s look at a very simple example
(Figure 2), inspired by [Russell/Norvig 2003](#refRN). Say we have three random variables, namely the weather
($W$), with values {`sunny`, `rainy`}, traffic ($T$), which can be {`normal`, `bad`, `terrible`} and whether we are
late for a meeting ($L$: {`true`, `false`}).

<font color="blue"><a name="q2"></a>**QUESTION**: What are the types of the random variables $W$, $T$, and $L$?[$^2$](#answer2)</font>

From pop culture we know that it never rains in southern California, and from our meterological data
(see [section 2.1.1](#secprob)) we know that *never* means 5%. So the probabilities for $W$ are $(0.95, 0.05)$. If we
talk about the probability of a specific outcome of the variables values, we write $P(W =$ `sunny` $) = 0.95$
or shorter $P($ `sunny` $) = 0.95$.

If it rains, traffic tends to get worse, and if traffic is bad, we are more likely to be late for our meeting.
If it is sunny, the traffic behaves different than when it is rainy, so we have to specify the probability of
each value of $T$ for each value of $W$. We do that in a table, where each column is a value for a variable,
$T$ and $W$. Notice that the rows with the same value forW have to sum up to $1.0$. You can imagine that
each value for weather is a state you are in, and the different values for $T$ are options you can choose
from. Some options are more likely than others, but all probability is distributed between them (thus
summing to $1.0$). You cannot choose something that is not there.

Whether I am late for a meeting ($L$) in turn depends on the state of the traffic ($T$), so we have to
specify another table with probabilities for each value of $L$ given each value of $T$. Again you can see
that with worse traffic, our chances of being late increase.
Using the Bayes Net, we can now compute how likely we are to be late if the weather is bad but
traffic is normal, and other interesting things.

<a name="secHMMs"></a>2.2.2 Hidden Markov Models
--

<img src="pics/hmm-example.png" width="300px"/>
<div align="center">*Figure 3: A Hidden Markov Model, resulting from connecting several of the Bayes Net above in a sequence*</div>

Things change over time, but they might be connected. Tomorrow’s weather does not just happen, it
actually depends on the weather of today. If we want to capture this, we can include another kind of
conditional probabilities, namely the ones expressing how a random variable changes over time. This
is the Markov part of HMMs. To make things easier, we assume that each state depends only on the
previous one, not all previous states. This is one of the so-called ***Markov properties***. In order to 
make it a hidden Markov model, we assume that the random variable we are actually interested in is
unobservable, but related to something we can observe.

Using our example from above, we have the following scenario: one year from now, we want to get
the sequence of sunny and rainy days that occurred (see Figure 3). We do not remember the weather
($W$ is hidden), but we do have our diary, in which we noted for each day whether we were late or not
($L$ is our observed variable, and it is dependent on $W$). We just copy the Bayes net from above for
each day, and add the new transition probabilities $P(W_t | W_{t-1})$ between each of the Bayes nets. The
probability means “how likely is it to be {`rainy`, `sunny`} today if it was {`rainy`, `sunny`} yesterday".

In this case, we do have another hidden variable, $T$, but it is not necessary for HMMs in general. In most applications (such as the tagging we will see later), you only have one hidden layer. The important
part is that whether I am late one day does not depend on whether I was late the day before, but on the
*weather* on that day. Also, the traffic of today is independent of yesterday’s traffic. This is why there
are no arcs between the $T$ and $L$ variables, only the $W$ nodes. This is another Markov property, that the
observations (here, $L$) are independent of one another. We guesstimated the probabilities $P(T|W)$ and
$P(L|T)$ based on intuitions or data (we will later say, we *initialized* them), and we could now use EM
to adjust them to reflect observations, using our diary as data and reconstructing the weather one year
ago.

<a name="secexample"></a>3 Example Uses of EM
==

Let’s start with a practical NLP example of what EM is good for. Say you have a context free grammar
(CFG), and would like to attach probabilities to each rule like $P(S\ \to\ NP\ VP|S)$ to say how likely it is
that $S$ goes to $NP\ VP$ (vs. that $S$ goes to $S\ CC\ S$).

<font color="blue"><a name="q3"></a>**QUESTION**: How do you get the rule probabilites?[$^3$](#answer3)</font>

That’s easy enough, but it presupposes that we do have some suitable treebank. What if we do not
have a treebank, only plain text? This case is far more common... No problem, if you just had some
rule probabilities, you could generate a treebank by applying the most likely rules to produce the text.

<font color="blue"><a name="q4"></a>**QUESTION**: What algorithm would you use to do that?[$^4$](#answer4)</font>

Oh wait, dangit! To get those rule probabilities, you’d have to have a treebank to collect them from.
But that’s what we wanted in the first place. This is a pretty circular problem! “If only I knew the rule
probabilities, I could make a treebank from text. If only I had a treebank, I could compute the rule
probabilities..."

EM can help us solve this problem. But how?

If you know $k$-means clustering, you already know how EM works! If not, don’t despair. 
<img src="pics/kmeans.png" width="450px"/>
<div align="center">*Figure 4: $k$-means clustering. Modified from [Bishop 2006](refBishop)*</div>

Here is an example: We want to divide the green points in Figure 4 into a red and a blue cluster. If
only we knew the cluster centroids, we could assign the data points to the closest clusters. If only we
knew which clusters the data points belong to, we could compute their centroids... Sounds somehow
familiar? Again, we have a circular problem. And here is how we gonna solve it:
We start by randomly placing cluster centroids on the graph (a). Then, we assign each data point to
a cluster (b).

<font color="blue"><a name="q5"></a>**QUESTION**: Based on the image, what do we use to assign the points to clusters?[$^5$](#answer5)</font>

Then, we compute the centers of those new clusters and move the centroids to that position (c).
As we can see, we alternate between assigning the points to clusters and computing new centroids.
Once the centroids stop moving around, we are done.

EM works similarly, and in fact is a “soft" version of $k$-means. Instead of assigning each point to
just one cluster (hard clustering), EM will attach a probability to the membership of a point in each
cluster ($P($ `cluster` $|$ `point` $)$). A data point can thus belong to several clusters (though with different
probabilities).

One the most well-known NLP tasks for EM is labeling unannotated text. We want to find labeling
$argmax_t P(t|w)$, i.e., given some words $\mathbf{w}$, what is the best tag sequence $\mathbf{t}$? Or: “If only I knew the right
tag sequence, I could compute their probabilities... If only I knew the tag probabilities, I could tag
the words." This is going to be our running example, and I will use $\mathbf{t}$ for a tag sequence and $\mathbf{w}$ for a
word sequence. The subscript $i$ denotes the position in the respective sequence, so $w_i$ is the $i$th word).

As mentioned in the first section, there are several algorithms to do EM (especially the E step), and
the one we will be using is the ***Forward-Backward*** (or Baum-Welch) algorithm for labeling (there are other
algorithms for other tasks).

<a name="secgoal"></a>4 The Goal
==

What we want from EM is the ***model parameters*** of our grpahical model (i.e., the conditional probability tables) that best explain the observed data. If we plot how 
well a specific model configuration explains the data over all possible parameter configurations, we get the
graph in Figure 5.
<img src="pics/localmaxima.png" width="450px"/>
<div align="center">*Figure 5: What we try to optimize with EM*</div>

The configuration we want is the one at the highest peak. As you can see, there are several smaller
peaks. These are local maxima (they are a bit of a hassle, but we come back to that later).

Let’s first look at how we get those parameters. There will be a few formulas involved, but
don’t be intimidated, they are easier than they look.

Remember that EM models a joint probability distribution $P(x,y)$. We can write that as

$$P(x,y) = P(y) \times P(x|y) = P(x) \times P(y|x)$$

The last two parts are the same because we don’t care about the order of $x$ and $y$. To see why that is
so, look at the diagram in Figure 6. 
<img src="pics/venn.png" width="200px"/>
<div align="center">*Figure 6: Venn diagram for $P(x,y)$*</div>

$P(x,y)$ is the grey area. To get that, we can just look at $x$ and the
part of it that overlaps with $y$. This is $P(x) \times P(y|x)$. You could call this very $x$-centric. If you don’t like that (and I won't blame you), you can get the same result by looking at $y$ and the part of it that overlaps with $x$. That would be $P(y) \times P(x|y)$, which is, as you can see, the same as $P(x) \times P(y|x)$.

In our case, however, we don’t want to model $x$s and $y$s, but the probability of a word and tag
sequence occurring together, $P(\mathbf{w}, \mathbf{t})$. So let’s substitute $x$ and $y$ for $\mathbf{w}$ and $\mathbf{t}$ and see what we get.
Something like

$$P(\mathbf{w},\mathbf{t}) = P(\mathbf{t}) \times P(\mathbf{w}|\mathbf{t}) = P(\mathbf{w}) \times P(\mathbf{t}|\mathbf{w})$$

If we translate this into English, it says that the probability of seeing the word and tag sequence together ($P(\mathbf{w}, \mathbf{t})$) is equal to the probability
of the tag sequence $P(\mathbf{t})$ times the probability that we generate the words from that tag sequence
($P(\mathbf{w}|\mathbf{t})$). It is also equal to the probability of seeing those words ($P(\mathbf{w})$) times the probability of
turning those words into that tag sequence ($P(\mathbf{t}|\mathbf{w})$). Both of these formulations are equivalent to $P(\mathbf{w}, \mathbf{t})$, and that fact will come in handy.

What we want to maximize is the last part, $P(\mathbf{t}|\mathbf{w})$ (i.e., what is the best tag sequence for the sentence
we see), since we already have the words and want to know the tags. Let’s take the last two parts and move a
few things around to get $P(\mathbf{t}|\mathbf{w})$ alone.

<font color="blue"><a name="q6"></a>**QUESTION**: What operations do we have to do to get $P(\mathbf{t}|\mathbf{w})$? (Don't peek ahead...)[$^6$](#answer6)</font>

$$\frac{P(\mathbf{t}) \times P(\mathbf{w}|\mathbf{t})}{P(\mathbf{w})} = P(\mathbf{t}|\mathbf{w})$$

Ok, that’s something. Take a deep breath and make sure you followed this.
Since we observe $\mathbf{w}$ (the sentence), we can say that $P(\mathbf{w})$ is $1.0$. In that case we can forget
about that denominator (anything divided by $1.0$ stays the same as it was)! So what we want to optimize ($P(\mathbf{t}|\mathbf{w})) becomes simply

$$P(\mathbf{t}|\mathbf{w}) = P(\mathbf{t}) \times P(\mathbf{w}|\mathbf{t})$$

Much cleaner, hmm?

Let’s look at that P(\mathbf{t}). It's a sequence of tags, where each depends on the previous one. The probability of seeing the whole tag sequence $t_1, t_2, \cdots t_n$ (i.e., P(\mathbf{t})) is therefore really
just the product of seeing each of the tags following another tag. We can write that as

$$P(\mathbf{t}) = P(t_1) \times P(t_2|t_1) \times P(t_3|t_2) \times \cdots \times P(t_n|t_{n-1})$$

or for short

$$P(\mathbf{t}) = P(t_1) \times \prod^n_2 P(t_i|t_{i-1})$$

Ok, so we played with the formula, made it cleaner, and dissected $P(\mathbf{t})$. But how does that help us?
Good question...

We wanted to find the parameters for our model, and now we have them: $P(\mathbf{t})$ and $P(\mathbf{w}|{t})$. So we
put it all together, and what we want to optimize is finally the product of our two parameters.

$$ P(\mathbf{t}|\mathbf{w}) = P(t_1) \times P(w_1|t_1) \times \prod^n_2 P(t_i|t_{i-1}) \times P(w_i|t_i)$$

By the way: parameters come in two flavors: ***free parameters*** (these are the ones we want EM to 
optimize) and ***fixed parameters*** (we already know these and don’t want EM to change them). You can 
make all parameters free and let EM handle them, or just some!

<font color="blue"><a name="q7"></a>**QUESTION**: What would happen if you fixed all parameters?[$^7$](#answer7)</font>

<a name="secimplementation"></a>5 Implementation Details
==

In order to compute the parameters, we have to develop a data structure that allows us to manipulate
them. We will use a Hidden Markov Model to represent the model and ***lattice-based dynamic
programming*** to compute and manipulate the probabilities. The following sections walk through the
individual parts and explain them in detail using pseudocode. You can find a Python implementation
(not optimized for performance) [further down](#secrunnable).

5.1 HMM as Lattice
--

The idea behind our HMM is something like this: what we can see (the words) was generated by something we
cannot see (the tag sequence). This is our ***generative story*** (see Figure 7). Sounds strange? Just wait... 
<img src="pics/hmm.png" width="200px"/>
<div align="center">*Figure 7: The Hidden Markov Model of the POS tagging task*</div>


Tags are connected by ***transition probabilities*** $P(t_i|t_{i-1})$, and emit words with ***emission probabilities*** $P(w_i|t_i)$. These look strangely familiar... 
We could also say “Assume the first word was generated by the first tag, how likely is it the next word was generated by the next tag, and how likely is it that that tag followed the first?" This translates to a sequence of conditional probabilities that we already know from earlier (aren’t you glad now we went through all those equations?):

$$ P(\mathbf{t}|\mathbf{w}) = P(t_1) \times P(w_1|t_1) \times \prod^n_2 P(t_i|t_{i-1}) \times P(w_i|t_i)$$

Again, this just means “the most likely tag sequence given a sentence is computed by concatenating
the most likely tags that can emit those words".

<font color="blue"><a name="q8"></a>**QUESTION**: Why do we need $P(t_i|t_{i-1})$ in there? Why don’t we just take the best tag for each word ($P(t|w)$) and be done?[$^8$](#answer8)</font>

We can model HMMs as a matrix/***lattice*** (or automaton) of tags and words, as in Figure 8, by 
replacing each random variable in the HMM with all possible values and drawing all possible arcs
between them. This can be tricky, and translating from a graphical model to a lattice takes some
getting used to.
<img src="pics/lattice.png" width="600px"/>
<div align="center">*Figure 8: The lattice for the sentence "Mice like cheese" and two possible tags, N and V*</div>


In this case, we want to label “Mice like cheese", and have an alphabet of only two tags, `N` and
`V`. It is important to specify all the tags you want to use. We start from a designated start state and
from there choose one of the tags with the respective probability $P(t)$. From each of those possible tag
states, we can emit a word with the respective probability $P(w|t)$. Those are the horizontal lines in our
lattice. Then, we choose the next tag with some probability $P(t_i|t_{i-1})$. Those are the crossing lines in
the lattice.

To visualize this, we list all tags as rows. If we have a ***dictionary*** that tells us that some words 
can only have certain tags, we simply set all other $P(w|t)$ for this word to $0$ (here, we could set
($P($ `mice` | `verb` $) = 0$) and omit the arcs. If we don’t have a such a dictionary, we have to assume that all
words can be emitted by all tags and let EM figure it out.

Ultimately, we want to learn which tags follow one another, say `V` usually comes after `N`, and
which words have which tags, e.g., like is most of the times a verb, never a noun. Only expressed as
probabilities: what is $P($ `like`$|$ `V` $)$? So how do we assign those values? We reward good parameters, i.e.,
transitions that increase $P(sentence)$, and we decrease bad ones. As a first step, instead of just taking
whole counts of how often we see a transition, we "weigh" them by how likely the resulting sentence
was ($P(sentence)$). This is called ***fractional counts***.

<font color="blue"><a name="q9"></a>**QUESTION**: What probabilistic event do you think is describe by $P(sentence)$?[$^9$](#answer9)</font>

We could try to just generate all possible taggings of a sentence (see Figure 9) and count how often
we see `N` following `V` and “cheese" was tagged as `N`, and then sum them all up to get $P(sentence)$. Here, we have two tags and three words, so we get $2^3=8$ possible paths through the lattice. But there is a problem...
<img src="pics/naive.png" width="600px"/>
<div align="center">*Figure 9: Brute-force enumeration of all possible tag sequences*</div>

Say each word has on avg. $2.5$ tags, and a sentence has $17$ words, like this one. That’s $2.5^{17}$, or
$5,820,766$ possible paths through that lattice. For just one sentence! Imagine a sentence with 50
words... Clearly, we cannot afford to do that. We will have to do something else. And that something
is dynamic programming, see [Section 5.2](#secdp).

An aside: since we multiply a lot of transitions to get through the lattice, the numbers can
quickly become very small. To deal with this, you can use the logarithm of the probabilities.
The smaller a number gets, the larger its negative log will be. Since the range of probabilities is
between 0 and 1:0, this corresponds an interval between negative infinity and 0 in log world.
If you do use logarithms, all multiplications shown here become additions (which is slightly faster),
and all additions have to be log-additions, a special computation that unfortunately is relatively slow,
and works like this (adapted from [Manning/Schütze 2000](#refMS)):

```
m = min(x,y)
big = 10^30
if y-x > log(big):
    return (y)
else if x- y > log(big):
    return (x)
else 
    return (m + log(exp(x-m)+exp(y-m)))
```

**NB:** In the remainder of the notebook, I use “normal” probabilities (not logs), to de-clutter the notation, but in the [implementation](#secrunnable), we will use logs, and the `scipy` function `logaddexp()`.

5.2 Dynamic Programming
--

Back to our question: we wanted to know what $P($ `like` | `V` $)$ is. This now turns into the question of how
likely it is that we end up at the node that has $P($ `like` | `V` $)$ as outgoing transition (in our example node
(2,3)). And once we took that transition, what is the probability from the node we reach (i.e., (2,4))
to the end?

Since we have modeled the HMM as a lattice, we can use dynamic programming techniques. This
allows us to compute how likely it is to arrive at each node (with the Forward algorithm), and to get to
the end from there (with the Backward algorithm).

So this is where we use the Forward-Backward algorithm! It consists of two parts.

<font color="blue"><a name="q10"></a>**QUESTION**: Take a wild guess what the two parts are called.[$^{10}$](#answer10)</font>

We use Forward-Backward in order to efficiently compute for each sentence how often we see each
transition and what the probability of that sentence is. We need both for the fractional counts. Forward-
Backward is the E step in our EM implementation: we compute the expected counts given the current
model parameters.

<font color="blue"><a name="q11"></a>**QUESTION**: What was the E-step in clustering?[$^{11}$](#answer11)</font>

When you model the lattice, it is a good idea to use a matrix or some such data structure in your
implementation, so you can access the nodes directly. In the following, we’ll use a matrix with two
columns for each word, to make everything easier to see. In practice, people use one column per word and later
tease out the fractional counts for each type of parameter.

**The Forward Algorithm **
<img src="pics/alphalattice.png" width="600px"/>
<div align="center">*Figure 10: Computing the alphas in the Forward pass*</div>

In the forward pass, we compute a new lattice with the same dimensions as the original one, which forward pass
contains for each node the sum of all possible paths that lead up to there (see Figure 10). These values
are also called alphas. $\alpha[i, j]$ denotes the probability of all paths up to node $(i, j)$ (i.e., assigning tag $i$ to word $j$). 

$\alpha[START]$ is always $1.0$. Each subsequent $\alpha$ is just the sum of all transitions arriving there, each multiplied by the $\alpha$ of the node where it (the transition) originated.

$\alpha[END]$ is the sum of all paths through the lattice, which is equal to $P(sentence)$. $P(data)$ is the
sum of all $P(sentence)$ in the data. In each iteration, just add up all the $\alpha[END]$ of the sentences.
Remember, $P(data)$ has to increase with each iteration, or there is something wrong! EM guarantees
that the likelihood of the data increases at each iteration over the data. Outputting $P(data)$ is thus a
good way of debugging your code: if it does not increase, something went wrong...

In [13]:
def forward(alpha, tags, emissions, transitions, example):
    """
    forward pass
    @param alpha: initialized matrix. alpha[i,j] = the probability of ending up with tag i at word j
    @param tags: tag set
    @param emissions: emission parameters, nested dictionary
    @param transitions: transition parameters, nested dictionary
    @param example: list of words
    @return: alpha
    """    
    for i, tag in enumerate(tags):
        if tag in start:
            alpha[i][0] = start[ tag ]
            if example[0] in emissions[tag]:
                alpha[i][1] = alpha[i][0] + emissions[tag][ example[0] ]

    for j in range(2, N, 2):
        for i, tag1 in enumerate(tags):
            for k, tag2 in enumerate(tags):
                if tag1 in transitions[tag2]:
                    # alpha[i,j] += P(t1|t2) * alpha[k, j-1]
                    if alpha[i, j] == NINF:
                        alpha[i,j] = transitions[tag2][tag1] + alpha[k,j-1]
                    else:
                        alpha[i,j] = logadd(alpha[i][j],
                                            transitions[tag2][tag1]+alpha[k,j-1])
            if example[int(j/2)] in emissions[tag1]:
                # alpha[i, j+1] = alpha[i, j] * P(word_j|tag1)
                alpha[i,j+1] = alpha[i,j] + emissions[tag1][ example[int(j/2)] ]

    return alpha

**The Backward Algorithm**
<img src="pics/betalattice.png" width="600px"/>
<div align="center">*Figure 11: Computing the betas in the Backward pass*</div>

The backward pass is almost the same as the forward pass, just backwards (note how the direction of backward
the arrows is reversed in Figure 11). Again, we compute a new lattice, which contains for each node pass
the sum of all possible paths that lead from that node to the end. These values are called ***betas***. $\beta[i, j]$
denotes the summed probability of all paths from node $(i, j)$ to the end. This time, however, we start at
the end. $\beta[END]$ is always $1.0$.

A useful property for debugging is the fact that $\beta[START] = \alpha[END]$.

<font color="blue"><a name="q12"></a>**QUESTION**: Why are the two qunatities the same?[$^{12}$](#answer12)</font>

In [14]:
def backward(beta, tags, emissions, transitions, example):
    """
    backward pass
    @param alpha: initialized matrix
    @param tags: tag set
    @param emissions: emission parameters, nested dictionary
    @param transitions: transition parameters, nested dictionary
    @param example: list of words
    @return: completed beta
    """    
    # initialize beta starting from the back
    for i, tag in enumerate(tags):
        beta[i][-1] = sp.log(1.0)
        if example[-1] in emissions[tag]:
            beta[i][-2] = beta[i][-1] + emissions[tag][ example[-1] ]

    for j in range(N-3,0,-2):
        for i, tag1 in enumerate(tags):
            for k, tag2 in enumerate(tags):
                if tag2 in transitions[tag1]:
                    if beta[i,j] == NINF:
                        beta[i,j] = transitions[tag1][tag2] + beta[k,j+1]
                    else:
                        beta[i,j] = logadd(beta[i,j], transitions[tag1][tag2] + beta[k,j+1])
            if example[int(j/2)] in emissions[tag1]:
                beta[i, j-1] = beta[i, j] + emissions[tag1][example[int(j/2)]]

    return beta

**Collecting fractional counts**
<img src="pics/fraccount.png" width="300px"/>
<div align="center">*Figure 12: Collecting fractional counts for $P($ `like` | `V` $)$*</div>

Once we have the alphas and betas, it is easy to compute for each transition how much it contributes
to $P(sentence)$. So, once more, with conviction: how good is $P($ `like` | `V` $)$? Remember, we have to know
the likelihood of all possible paths arriving at node $(2,3)$, and the probability – once we have taken the
transition – from node $(2,4)$ to the end. See Figure 12.

We used the forward algorithm to get the probability of arriving at node $(2,3)$, and the backward
algorithm to compute how likely it is from node $(2,4)$ to the end. We divide that by the likelihood of
the sentence ($= \alpha[END]$), et voila!

In [15]:
def collect_counts(alpha, beta, tags, start, start_counts, emissions, emission_counts, transitions, transition_counts, example, example_likelihood):

    for i, tag1 in enumerate(tags):
        if tag1 in start:
            # if no smoothing, start from scratch, otherwise just add up
            if start_counts[tag1] == sp.log(0.0):
                start_counts[tag1] = start[tag1] + beta[i, 0] - example_likelihood
            else:
                start_counts[tag1]=logadd(start_counts[tag1],
                                          start[tag1]+beta[i,0]-example_likelihood)

        for j, word in enumerate(example):
            if word in emissions[tag1]:
                if emission_counts[tag1][word] == sp.log(0.0):
                    emission_counts[tag1][word] = alpha[i, j*2] + emissions[tag1][word] + beta[i, (j*2)+1] - example_likelihood
                else:
                    emission_counts[tag1][word]=logadd(emission_counts[tag1][word], alpha[i, j*2] + emissions[tag1][word] + beta[i, (j*2)+1] - example_likelihood)

            for k, tag2 in enumerate(tags):
                if j < len(example)-1 and tag2 in transitions[tag1]:
                    if transition_counts[tag1][tag2] == sp.log(0.0):
                        transition_counts[tag1][tag2] = alpha[i, j*2+1] + transitions[tag1][tag2] + beta[k, (j+1)*2] - example_likelihood
                    else:
                        transition_counts[tag1][tag2]=logadd(transition_counts[tag1][tag2], alpha[i, j*2+1] + transitions[tag1][tag2] + beta[k, (j+1)*2] - example_likelihood)
   
    return start_counts, transition_counts, emission_counts

**The M Step**

Computing alphas and betas and collecting the fractional counts for all free parameter transitions over
all examples is the ***E step***. This, as the name suggests, is one half of Forward-Backward EM, and in 
this case the bigger half. The ***M step*** is comparatively trivial: after having gone through all the data, we 
just normalize our fractional counts to get probabilities back (remember, probabilities are just normalized
counts).

<font color="blue"><a name="q13"></a>**QUESTION**: What do you need to compute conditional probabilities from counts?[$^{13}$](#answer13)</font>

In [16]:
def normalize_counts(start_counts, start, emission_counts, emissions, transition_counts, transitions):
    start_count_total = reduce(logadd, list(start_counts.values()))

    print("\nNew START probabilities:")
    print(('-' * 20))
    for tag in start:
        start[tag] = start_counts[tag] - start_count_total
        print(('P(%s) = %.2f' % (tag, sp.exp(start[tag]))))

    print("\nNew EMISSION probabilities:")
    print(('-' * 20))
    for tag, words in emissions.items():
        emission_tag_total = reduce(logadd, list(emission_counts[tag].values()))
        for word in words:
            emissions[tag][word] = emission_counts[tag][word] - emission_tag_total
            print(('P(%s|%s) = %.2f' % (word,tag, sp.exp(emissions[tag][word]))))

    print("\nNew TRANSITION probabilities:")
    print(('-' * 20))
    for tag, tag_successors in transitions.items():
        transition_tag_total = reduce(logadd, list(transition_counts[tag].values()))
        for tag_successor in tag_successors:
            transitions[tag][tag_successor] = transition_counts[tag][tag_successor] - transition_tag_total
            print(('P(%s|%s) = %.2f' % (tag_successor,
                                        tag,
                                        sp.exp(transitions[tag][tag_successor]))))

    print()
    return start, emissions, transitions

5.3 Putting It All Together
--

If we put the E step and the M step together, we end up with the Forward-Backward EM algorithm!

The convergence criterion in this case is how much the data likelihood P(data) has improved since
the last iteration. Once it starts to flatten out, we can assume that we reached a maximum on our curve
and stop. It is also customary to set a maximum number of iterations (50) and stop even if EM has not
converged, to avoid overfitting.

And that’s it. No magic, no silver bullets, just counting and normalizing. EM will adjust all
the free parameters to get the maximum data likelihood, and you can then use those probabilities to
label data using the Viterbi algorithm.

<a name="secrunnable"></a>5.4 Example Run
--

If you want to check whether your implementation is working, here is a little worked toy example. Let’s take
the tags `N` and `V`, and the words *can* and *I* and initialize our model with the following numbers
(this example is based on one used by Kevin Knight.)

Transitions:

$P($ `V` | `V` $) = 0.6$

$P($ `N` | `V`$) = 0.4$

$P($ `V` | `N`$) = 0.9$

$P($ `N` | `N`$) = 0.1$

$P($ `V` | `START` $) = P($ `V` $) = 0.6$

$P($ `N` | `START` $) = P($ `N ` $) = 0.4$


Emissions:

$P($ `can` | `V` $) = 0.5$

$P($ `I` | `V` $) = 0.5$

$P($ `can` | `N` $) = 0.5$

$P($ `I` | `N` $) = 0.5$


As our only observed instance we use the sentences "*can I can*" and "*I can can*".

In [17]:
start = defaultdict(lambda: LINF)
# P(x|y) = emissions[y][x] - this allows to sum over all values given y
emissions = defaultdict(lambda: defaultdict(lambda: LINF))
transitions = defaultdict(lambda: defaultdict(lambda: LINF))

# initialize
start['V'] = sp.log(0.6)
start['N'] = sp.log(0.4)

emissions['V']['I'] = sp.log(0.5)
emissions['V']['can'] = sp.log(0.5)
emissions['N']['I'] = sp.log(0.5)
emissions['N']['can'] = sp.log(0.5)

transitions['V']['V'] = sp.log(0.6)
transitions['V']['N'] = sp.log(0.4)
transitions['N']['N'] = sp.log(0.1)
transitions['N']['V'] = sp.log(0.9)

tags = ['V', 'N']

examples = [
    ['can', 'I', 'can'],
    [ 'I', 'can', 'can']
]

The actual code is two simple loops (one over the data, one until convergence) that call our functions defined above. You can run it and observe the output. Your $\alpha$s and $\beta$s for "*can I can*" after the first iteration should look like they do in Figure 13, where the $\alpha$ values for each node are shown on a white
background above the nodes, and $\beta$ values with a grey background below the nodes. The arrows point
forward, but you can just ignore them.

<img src="pics/example_run.png" width="600px"/>
<div align="center">*Figure 13: $\alpha$ (unshaded) and $\beta$ (shaded boxes) values for example run*</div>

In [18]:
converged = False
old_data_likelihood = sp.log(0.0)
iteration = 1

# repeat until convergence
# (i.e., difference of data-log likelihood < THRESHOLD or MAX_NUM_ITERATIONS)
while not converged:
    print(('=' * 20))
    print(("iteration %s" % (iteration)))
    print(('=' * 20))
    data_likelihood = sp.log(1.0)

    # clear counts
    start_counts = defaultdict(lambda: SMOOTHING)
    emission_counts = defaultdict(lambda: defaultdict(lambda: SMOOTHING))
    transition_counts = defaultdict(lambda: defaultdict(lambda: SMOOTHING))

    ##########
    # E-Step #
    ##########
    for example in examples:
        starttime = time.time()
        N = len(example) * 2
        M = len(tags)

        ################
        # forward pass #
        ################
        alpha = forward(sp.ones((M,N)) * NINF, tags, emissions, transitions, example)
        print("ALPHA:")
        print((sp.exp(alpha)))
        
        #################
        # backward pass #
        #################
        beta = backward(sp.ones((M,N)) * NINF, tags, emissions, transitions, example)
        print("BETA:")
        print((sp.exp(beta)))
        print()

        # sum of all paths through example
        example_likelihood = reduce(logadd, alpha[:, -1])
        print(("example likelihood: ", sp.exp(example_likelihood)))

        #############################
        # collect fractional counts #
        #############################
        start_counts, transition_counts, emission_counts = collect_counts(alpha, beta, tags, start, start_counts, emissions, emission_counts, transitions, transition_counts, example, example_likelihood)

        # add example likelihood to data likelihood
        data_likelihood += example_likelihood

        print(("%.5fsec" % (time.time()-starttime)))
    print()

    ############################
    # M-Step: normalize counts #
    ############################
    start, emissions, transitions = normalize_counts(start_counts, start, emission_counts, emissions, transition_counts, transitions)

    change = -(sp.exp(old_data_likelihood) - sp.exp(data_likelihood))
    assert change > 0.0
    print("CHANGE:")
    print(('old: %.5f (%s)' % (sp.exp(old_data_likelihood), old_data_likelihood)))
    print(('new: %.5f (%s)' % (sp.exp(data_likelihood), data_likelihood)))
    print("change: ", change)
    print('*' * 50)
    print()
    if change <= THRESHOLD or iteration > MAX_NUM_ITERATIONS:
        converged = True
        print('Converged!')

    old_data_likelihood = data_likelihood
    if iteration == 10:
        break
        
    iteration += 1


iteration 1
ALPHA:
[[ 0.6    0.3    0.36   0.18   0.171  0.085]
 [ 0.4    0.2    0.14   0.07   0.079  0.039]]
BETA:
[[ 0.125  0.25   0.25   0.5    0.5    1.   ]
 [ 0.125  0.25   0.25   0.5    0.5    1.   ]]

('example likelihood: ', 0.12499999999999997)
0.00124sec
ALPHA:
[[ 0.6    0.3    0.36   0.18   0.171  0.085]
 [ 0.4    0.2    0.14   0.07   0.079  0.039]]
BETA:
[[ 0.125  0.25   0.25   0.5    0.5    1.   ]
 [ 0.125  0.25   0.25   0.5    0.5    1.   ]]

('example likelihood: ', 0.12499999999999997)
0.00112sec


New START probabilities:
--------------------
P(V) = 0.59
P(N) = 0.41

New EMISSION probabilities:
--------------------
P(I|V) = 0.34
P(can|V) = 0.66
P(I|N) = 0.36
P(can|N) = 0.64

New TRANSITION probabilities:
--------------------
P(V|V) = 0.59
P(N|V) = 0.41
P(V|N) = 0.85
P(N|N) = 0.15

CHANGE:
old: 0.00000 (-inf)
new: 0.01562 (-4.15888308336)
change:  0.015625
**************************************************

iteration 2
ALPHA:
[[ 0.591  0.392  0.456  0.154  0.151  0.1  ]

/Users/dirkhovy/anaconda/envs/py3.4/lib/python3.4/site-packages/numpy/lib/scimath.py:262: RuntimeWarning: divide by zero encountered in log
  return nx.log(x)


<a name="secfinally"></a>6 Finally...
==

CONGRATULATIONS! You have made it through. You now know EM and can go and try it out...
You do not have to be constrained to HMMs with hidden inputs, like we used here. Maybe
you do not have sequential data. In that case the HMM becomes a Bayesian network, which is like an
HMM without the transitions ([Hovy et al. (2013)](#refMACE) is an example for training a Bayes net with EM to
find out which annotators are more reliable than others, and what the most likely answer is).
Maybe you know both the inputs and outputs, but not the hidden variable $X$ that connects them .
In that case you just want to know the conditional probabilities $P(X|input)$ and $P(output|X)$ (in our
diary example above, you might also have the weather reports from that time and want to compute the
traffic probabilities). EM can help you in those cases, too.

If you want to apply EM to other problems, always start with the inputs and outputs. What are
the observed variables? Are the inputs hidden, or can they be observed, too? Is it a sequential model
or a network? Write down the joint probability $P(x,y)$ and see how you can factor it. Drawing a
graphical model helps.

Think about how you can express the parameters as conditional probabilities. Do you need additional
(hidden) variables? What is your E step, what is your M step, what is the data you iterate
over? Once you have figured all this out, you can start to look into implementations. Do not focus on
implementation details like Forward-Backward in the beginning.

<a name="sectrouble"></a>7 Troubleshooting
==

There are some problems that can arise, and it is good to be aware of them and know how to deal with,
or, better, avoid them.

- **Problem**: What we maximize – $P(data)$ – is often not what we want to evaluate (say, the tagging
accuracy as compared to gold data).

- **Solution**: Design your model with this in mind and try to formulate the problem so that the two
criteria are similar. Keep test data around and evaluate your models on it.


- **Problem**: ***Local maxima***. EM improves P(data) at each iteration, but can get stuck in local 
maxima. Remember the graph from section 4: there are many suboptimal parameter configurations at
which EM stops because it can not improve from there. The result is not the best model, though.

- **Solution**: Restart EM 50 times or more with random initializations, and remember the model that
got the best data likelihood. Each time you restart, you start at a different point along the curve, and
hopefully eventually at one that leads to the global optimum.


- **Problem**: EM does not care about semantics! Whether a label makes sense or not is irrelevant,
as long as it explains the data! It uses this ***weird tag*** that only occurs once or twice, like `FW` (foreign 
word).

- **Solution**: Use a dictionary that constrains EM’s $P(\mathbf{w}|\mathbf{t})$ choices to possible ones (all others will be
0). If you do completely unsupervised training (no dictionary), the labels become meaningless, and
the task is more like clustering. Each label becomes a cluster. In this case, you have to afterwards map
each of the clusters to a tag in order to label data and get the accuracy. One mapping is many-to-1 
(see [Johnson 2007](#refJohnson)). Also, keep your data clean. EM wants to use everything. Yes, also that weird tag
that got in by accident... By assigning it to a frequent word, EM actually even thinks it has done a
good job.


- **Problem**: EM uses unlikely tags too frequently, i.e. we have a rather ***flat label distribution***. 
In language, however, most probability mass should go to one or two cases, the rest becomes less and 
less likely, i.e., we have a Zipfian distribution (see below).

- **Solution**: Use smoothing and a dictionary. Additionally, you might want to try techniques like
L$_0$ normalization ([Vaswani/Pauls/Chiang 2010](#refVaswani)) or Bayesian inference with Gibbs sampling and sparse
priors.
<img src="pics/distribution.png" width="350px"/>
<div align="center">*Figure 14: Difference between label distribution in EM and in reality*</div>



- **Problem**: EM changes good initial parameters to make them worse. 

- **Solution**: Fix as many parameters as you can! That will guide EM to only optimize the right 
things. Also, add ***pseudo-counts*** before normalization to make changes less dramatic. The higher the
pseudo-counts, the smaller the normalization effect. You can see in Table 1 how the normalized values
of two parameters change when using different pseudo-counts. If you do not want to fix anything
random restarts can help to find a good starting point.
<img src="pics/smoothing.png" width="400px"/>
<div align="center">*Table 1: Influence of pseudo-counts on parameters*</div>


- **Problem**: The resulting model is ***overfitting*** the training data (“Look, I can explain this data 
perfectly! And nothing else. . . ").

- **Solution**: Again, use ***smoothing*** (add $n$ to the fractional counts before normalizing) and stop after a
maximum amount of iterations (usually 50). This will prevent the data from being exactly modeled.


- **Problem**: Ties. All transition options leaving from one node are equally good. EM doesn’t 
take a stance and just leaves all of them as is.

- **Solution**: Start out randomly to avoid ties, and do restarts. This way, you can break the ties.

<a name="secreading"></a>8 Useful Reading
==

If you want to read the original, go for [Dempster/Laird/Rubin (1977)](#refDempster).
[Rabiner/Juang (1986)](#refRabiner) is a general overview over ***parameter estimation***, but very math-heavy. [Manning/Schütze (2000)](#refMS) has a chapter on EM, based on clustering, but with an eye on other NLP applications.
One of the most famous implementations for NLP is the tagging paper by [Merialdo (1994)](refMerialdo),
which also gives you a good idea about the various parameters you can set.

If you want to know more about the ***Forward-Backward*** procedure of calculating alphas and betas,
check out Jason Eisner’s tutorial ([Eisner, 2002](#refEisner)), including a spreadsheet with the changing values.
The second edition of the AI handbook ([Russell/Norvig, 2003, 724 – 733](refRN)) has a comprehensive
section about EM.

Kevin Knight has written a very compelling ***introduction*** ([Knight, 2009a](#refKnightA)). It is mainly about
Bayesian Inference, but explains EM very nicely. You might also want to check out his tutorials
for Carmel ([Knight, 2009b](#refKnightB)), a software that helps you implement graphical models as automata and
train them with EM. Also, the workbook for MT ([Knight, 1999](#refKnightMT)) contains a useful section on EM.

[Vaswani/Pauls/Chiang 2010](#refVaswani) show how using ***L$_0$ normalization*** can lead to smaller models and a L0
sparser distribution, which improves language related tasks a lot, because it creates a more Zipfian normalization
distribution (see [Hovy et al. 2011](#refHovy) for another application).

<a name="secadvanced"></a>9 Advanced Topics
==

The goal of his section is mainly to give you an idea of some of the topics are out there to address
problems with EM. It would lead too far to go into detail for each, so we will give a very high-level
intuition and refer to related work for details.


9.1 Scaling
--

In the examples above, we have used logarithms to prevent underflow of the probabilities. That turned
all our multiplications into additions (which is good, because it’s fast), but it also turned every addition
into a function call to a log addition (which unfortunately is very slow). There is another way to prevent
underflow, called ***scaling***, but it requires some more bookkeeping. The idea is that we normalize each 
column in our lattice so that it sums to 1. That way, the probabilities won’t get too small, and we can
still use multiplication and normal addition.

Practically, we add a vector to our forward-backward procedure. It has the same length as out lattice,
and for each position of the lattice contains the sum of the forward probabilities at that position, the
***scaling factor***. Before we move on, we normalize the column by that factor, so that it sums to 1. The 
backward pass uses the same scaling parameters as the forward pass.
We can then use the scaled forward and backward matrices as well as the scaling factor vector to
compute the fractional counts. See [Shen (2008)](#refShen) for more details (abstract mathematical notation, but
very good explanations).


9.2 Variational Bayes
--

We have earlier seen the idea of adding pseudo-counts to the fractional counts, which helps with
smoothing. Variational Bayes inference works similarly, only that now we define a prior for our prior
parameters.

A ***prior*** is essentially a curve that looks similar to what we would like to achieve (see the example
above of how EM’s distribution differs from the real one: a prior would look like the real one). In
practice, the prior here is a ***Dirichlet distribution***, which takes two parameters: a probability distribution and a vector of shape parameters. Both have the same number of elements (or dimensions). If 
we have only two dimensions in each, the prior is called a Beta function. Typically, the probability
distribution are our transition parameters, and the shape parameters are something like pseudo-counts
for each element.

This sounds rather complicated, but is relatively easy to implement. The E-step stays the same
as before. In the M-step, the elements of the shape parameter vector are added as pseudo-counts
to the matching fractional counts, and the result is passed through a ***Digamma function*** and finally
exponentiated. To normalize, we add the sum of all elements in the shape vector to our denominator
and again run it though Digamma and exponentiation. This is like a softer version of smoothing with
pseudo-counts, where we have separate counts for each parameter.


9.3 Type and Token Constraints
--

So far, we have pretended that every word can be labeled with all tags. However, we know that that is
not true: for example, “I” can never be a preposition or an adverb. One way to encode that knowledge
is to add a ***dictionary***. For each word, it lists the allowed tags. Essentially, this sets all 
parameters for disallowed tags to 0. Using a dictionary encodes general type constraints for a word, type
i.e., the ***constraints*** hold whenever we see the word anywhere in our data. Dictionaries are either built 
by hand or derived from some annotated data. In our example, we could restrict the legal tags for the
word “I” to just nouns.

There is another type of constraints, namely ***token constraints***. A token is a specific word in a 
specific context, so these constraints only hold for certain words in certain contexts. In our example,
we just might happen to know that the first “can” is a verb. ***Type constraints*** can come from simple
rules (e.g., something like “any word after the is either an adjective or a noun”), manual annotations
(going through the data and tagging some sentences) or through transfer from another data set.

Both types of constraints basically do the same thing, though: they reduce the number of nodes for a word
in our lattice. Type constraints do that for all occurrences of the word, token constraints only for some.
When we have an actual annotation for a word, the number of nodes is reduced to one. This greatly
reduces the number of legal paths through a sentence, so not only does it make forward-backward
more efficient, it also guides the algorithm because you can now only collect fractional counts for
certain parameters. See [Täckström et al. (2013)](#refOscar) for examples of using type and token constraints in an
application.

<a name="secfurther"></a>10 Going Further...
==

If you want to get deeper into the matter, you could look into ***EM with features*** (Berg-Kirkpatrick et al., 2010). Instead of just using conditional probabilities, which often cannot capture 
useful properties (or only when encoded as additional states), you can add all the features you like in
discriminative models (like word suffixes or capitalization) and still do unsupervised learning.

Or explore ***structural EM***, which not only learns the parameter values, but also how many parameters there should be.

If you do any of these things, or if you have questions, comments, or criticism, send me a mail—I’d
be dead curious to know!

<a name="secacknowledgements"></a>Acknowledgements
==

Thanks to Kevin Knight and David Chiang for the basics, Congxing Cai, Karl-Moritz Hermann, and
Eduard Hovy for useful comments, and Ashish Vaswani, Victoria Fossum, and Taylor Berg-Kirkpatrick
for many enlightening discussions.

References<a name="secref"></a>
==

<a name="refTaylor"></a>BERG-KIRKPATRICK, TAYLOR ET AL. (2010): Painless Unsupervised Learning with Features. In: North American Chapter
of the Association for Computational Linguistics.

<a name="refBishop"></a>BISHOP, C.M. (2006): Pattern recognition and machine learning. New York: Springer.

<a name="refDempster"></a>DEMPSTER, ARTHUR P./LAIRD, NAN M./RUBIN, DONALD B. (1977): Maximum likelihood from incomplete data via the
EM algorithm. In: Journal of the Royal Statistical Society. Series B (Methodological), 39, Nr. 1, 1–38.

<a name="refEisner"></a>EISNER, JASON (2002): An interactive spreadsheet for teaching the forward-backward algorithm. In: Proceedings of the
ACL-02Workshop on Effective tools and methodologies for teaching natural language processing and computational
linguistics-Volume 1. Association for Computational Linguistics, 10–18.

<a name="refMACE"></a>HOVY, DIRK ET AL. (2013): Learning Whom to trust with MACE. In: Proceedings of NAACL HLT.

<a name="refHovy"></a>HOVY, DIRK ET AL. (2011): Unsupervised Discovery of Domain-Specific Knowledge from Text. In: Proceedings of the
49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies. Portland,
Oregon, USA: Association for Computational Linguistics, pages 1466–1475. [http://www.aclweb.org/anthology/P11-1147.pdf](http://www.aclweb.org/anthology/P11-1147.pdf)

<a name="refJohnson"></a>JOHNSON, MARK (2007): Why doesn’t EM find good HMM POS-taggers. In: Proceedings of the 2007 Joint Conference
on Empirical Methods in Natural Language Processing and Computational Natural Language Learning (EMNLPCoNLL).,
296–305.

<a name="refKnightMT"></a>KNIGHT, KEVIN (1999): A statistical MT tutorial workbook. JHU Summer Workshop, 1999.

<a name="refKnightA"></a>KNIGHT, KEVIN (2009a): Bayesian Inference with Tears. [http://www.isi.edu/natural-language/people/bayeswith-tears.pdf](http://www.isi.edu/natural-language/people/bayeswith-tears.pdf).

<a name="refKnightB"></a>KNIGHT, KEVIN (2009b): Training Finite-State Transducer Cascades with Carmel. 2009b.

<a name="refMS"></a>MANNING, C.D./SCHÜTZE, H. (2000): Foundations of statistical natural language processing. MIT Press.

<a name="refMerialdo"></a>MERIALDO, BERNARD (1994): Tagging English text with a probabilistic model. In: Computational linguistics, 20, Nr. 2,
155–171.

<a name="refRabiner"></a>RABINER, L./JUANG, B. (1986): An introduction to hidden Markov models. In: IEEE ASSp Magazine, 3, Nr. 1 Part 1, 4–16.

<a name="refRN"></a>RUSSELL, S.J./NORVIG, P. (2003): Artificial intelligence: a modern approach. 2nd edition. Upper Saddle River, NJ: Prentice
Hall.

<a name="refShen"></a>SHEN, DAWEI (2008): Some Mathematics for HMMs. [http://courses.media.mit.edu/2010fall/mas622j/ProblemSets/ps4/tutorial.pdf](http://courses.media.mit.edu/2010fall/mas622j/ProblemSets/ps4/tutorial.pdf) – visited on 3/21/2013.

<a name="refOscar"></a>TÄCKSTRÖM, OSCAR ET AL. (2013): Token and Type Constraints for Cross-Lingual Part-of-Speech Tagging. In: Transactions
of the ACL.

<a name="refVaswani"></a>VASWANI, ASHISH/PAULS, ADAM/CHIANG, DAVID (2010): Efficient optimization of an MDL-inspired objective function
for unsupervised part-of-speech tagging.

<a name="secanswers"></a>Answers
===

1. <a name="answer1"></a>$P(x|y) = \frac{P(x,y)}{P(y)}$ [back](#q1)

2. <a name="answer2"></a>$W$ is discrete, $T$ is discrete, but binary, and $L$ is boolean, and thus also binary. [back](#q2)

3. <a name="answer3"></a>We count the occurrences of each rule S!NP VP in a treebank and normalize by the number of times we have seen S. [back](#q3)

4. <a name="answer4"></a>Exactly, CKY parsing... You are getting good at this! [back](#q4)

5. <a name="answer5"></a>We compute the distance between the cluster centroids and each point and assign it to the closest centroid. [back](#q5)

6. <a name="answer6"></a>We divide $P(t) \times P(w|t) = P(w) \times P(t|w)$ by $P(w)$ [back](#q6)

7. <a name="answer7"></a>You already had your model and would not need EM... [back](#q7)

8. <a name="answer8"></a>Because words tend to be ambiguous. What is the most likely tag for "can"? It depends. It could be `NOUN`, `VERB`, or `AUX`. But what is the most likely tag if we see "the can"? Still `VERB` or `AUX`? Probably not... The context disambiguates it. That’s why we want the transition probabilities. [back](#q8)

9. <a name="answer9"></a>The probability that we end up with this sentence if we ran our model in generation mode. [back](#q9)

10. <a name="answer10"></a>Well, Forward and Backward... What did you think? Though you could of course call Forward "Baum" and Backward "Welch", but that seems a little unfair to Mr. Welch. [back](#q10)

11. <a name="answer11"></a>Assigning each data point to one of the centroids (= the current model parameters). [back](#q11)

12. <a name="answer12"></a>Because in both cases we walked through all possible paths of the lattice, summing them up. [back](#q12)

13. <a name="answer13"></a>The count of the two events together and the counts of the given part. [back](#q13)